In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 784).astype('float32') / 255.0
x_test = x_test.reshape(-1, 784).astype('float32') / 255.0

In [ ]:
weights = {
    'W1': tf.Variable(tf.random.normal([784, 256], stddev=0.1)),
    'b1': tf.Variable(tf.zeros([256])),
    'W2': tf.Variable(tf.random.normal([256, 128], stddev=0.1)),
    'b2': tf.Variable(tf.zeros([128])),
    'W3': tf.Variable(tf.random.normal([128, 64], stddev=0.1)),
    'b3': tf.Variable(tf.zeros([64])),
    'W4': tf.Variable(tf.random.normal([64, 10], stddev=0.1)),
    'b4': tf.Variable(tf.zeros([10]))
}

In [ ]:
def forward(x):
    Z1 = tf.matmul(x, weights['W1']) + weights['b1']
    A1 = tf.nn.relu(Z1)
    Z2 = tf.matmul(A1, weights['W2']) + weights['b2']
    A2 = tf.nn.relu(Z2)
    Z3 = tf.matmul(A2, weights['W3']) + weights['b3']
    A3 = tf.nn.relu(Z3)
    Z4 = tf.matmul(A3, weights['W4']) + weights['b4']
    return tf.nn.softmax(Z4)

In [ ]:
def train_with_tape(epochs=5, lr=0.01):
    for epoch in range(epochs):
        for x_batch, y_batch in tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32):
            with tf.GradientTape() as tape:
                preds = forward(x_batch)
                loss = tf.reduce_mean(
                    tf.keras.losses.sparse_categorical_crossentropy(y_batch, preds))

            grads = tape.gradient(loss, list(weights.values()))

            for (w, g) in zip(weights.values(), grads):
                w.assign_sub(lr * g)

In [ ]:
def train_with_fit():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=5, batch_size=32)
    return model

In [ ]:
tape_weights = train_with_tape()
fit_model = train_with_fit()

print("GradientTape Test Accuracy:",
      tf.reduce_mean(tf.metrics.sparse_categorical_accuracy(
          y_test, forward(x_test))).numpy())
print("model.fit Test Accuracy:", fit_model.evaluate(x_test, y_test, verbose=0)[1])